In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [3]:
train_df = pd.read_csv('./best_train.csv')
test_df = pd.read_csv('./best_test.csv')

In [4]:
best_cols = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5',
 'diff_pres_1_first',
 'diff_pres_1_min',
 'diff_pres_1_max',
 'range_pres_1',
 'scale_pres_1',
 'diff_ccsm3_month_1_first',
 'diff_ccsm3_month_1_min',
 'diff_ccsm3_month_1_max',
 'range_ccsm3_month_1',
 'scale_ccsm3_month_1',
 'diff_sst_1_month_1_first',
 'diff_sst_1_month_1_min',
 'diff_sst_1_month_1_max',
 'range_sst_1_month_1',
 'scale_sst_1_month_1',
 'sst_1_lag_1',
 'sst_5_lag_1',
 'sst_4_lag_1',
 'sst_1_lag_1_season',
 'sst_5_lag_1_season',
 'sst_4_lag_1_season']
target=["contest-tmp2m-14d__tmp2m"]

In [ ]:
X = train_df[best_cols].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=300000, verbose=200, task_type="GPU",
                           devices='0:1', learning_rate = 0.03)
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=1000, early_stopping_rounds=200)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[best_cols].values))
s = 0
for pred in preds:
    s += pred
s = s/5
res_df = pd.read_csv('./sample_solution.csv')
res_df['contest-tmp2m-14d__tmp2m'] = s
res_df.to_csv('submission_more_trees.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:19,  3.95s/it]


(375734, 137) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

0:	learn: 9.6192707	test: 9.5977475	best: 9.5977475 (0)	total: 51ms	remaining: 4h 15m 7s
1000:	learn: 0.8919410	test: 0.9001836	best: 0.9001836 (1000)	total: 5.88s	remaining: 29m 16s
2000:	learn: 0.6762102	test: 0.6888285	best: 0.6888285 (2000)	total: 11.5s	remaining: 28m 31s
3000:	learn: 0.5744856	test: 0.5894723	best: 0.5894723 (3000)	total: 16.9s	remaining: 27m 51s
4000:	learn: 0.5076914	test: 0.5245689	best: 0.5245689 (4000)	total: 22.1s	remaining: 27m 15s
5000:	learn: 0.4610099	test: 0.4796216	best: 0.4796216 (5000)	total: 27.6s	remaining: 27m 6s
6000:	learn: 0.4242731	test: 0.4444613	best: 0.4444613 (6000)	total: 32.9s	remaining: 26m 49s
7000:	learn: 0.3954657	test: 0.4171923	best: 0.4171923 (7000)	total: 38.1s	remaining: 26m 34s
8000:	learn: 0.3722375	test: 0.3951681	best: 0.3951681 (8000)	total: 43.4s	remaining: 26m 25s
9000:	learn: 0.3524618	test: 0.3767130	best: 0.3767130 (9000)	total: 48.7s	remaining: 26m 14s
10000:	learn: 0.3353633	test: 0.3607210	best: 0.3607210 (10000)	to

86000:	learn: 0.1096961	test: 0.1625038	best: 0.1625038 (86000)	total: 7m 42s	remaining: 19m 10s
87000:	learn: 0.1089765	test: 0.1619476	best: 0.1619476 (87000)	total: 7m 47s	remaining: 19m 5s
88000:	learn: 0.1082691	test: 0.1614155	best: 0.1614155 (88000)	total: 7m 53s	remaining: 18m 59s
89000:	learn: 0.1075650	test: 0.1608728	best: 0.1608728 (89000)	total: 7m 58s	remaining: 18m 54s
90000:	learn: 0.1068960	test: 0.1603636	best: 0.1603636 (90000)	total: 8m 3s	remaining: 18m 48s
91000:	learn: 0.1062175	test: 0.1598558	best: 0.1598558 (91000)	total: 8m 9s	remaining: 18m 43s
92000:	learn: 0.1055578	test: 0.1593463	best: 0.1593463 (92000)	total: 8m 14s	remaining: 18m 37s
93000:	learn: 0.1049033	test: 0.1588535	best: 0.1588535 (93000)	total: 8m 19s	remaining: 18m 32s
94000:	learn: 0.1042649	test: 0.1583672	best: 0.1583672 (94000)	total: 8m 24s	remaining: 18m 26s
95000:	learn: 0.1036382	test: 0.1579028	best: 0.1579028 (95000)	total: 8m 30s	remaining: 18m 21s
96000:	learn: 0.1030296	test: 0.1

...

86000:	learn: 0.1096821	test: 0.1613893	best: 0.1613893 (86000)	total: 7m 42s	remaining: 19m 10s
87000:	learn: 0.1089738	test: 0.1608446	best: 0.1608446 (87000)	total: 7m 47s	remaining: 19m 5s
88000:	learn: 0.1082722	test: 0.1603142	best: 0.1603142 (88000)	total: 7m 53s	remaining: 18m 59s
89000:	learn: 0.1075875	test: 0.1597976	best: 0.1597976 (89000)	total: 7m 58s	remaining: 18m 53s
90000:	learn: 0.1069005	test: 0.1592715	best: 0.1592715 (90000)	total: 8m 3s	remaining: 18m 47s
91000:	learn: 0.1062411	test: 0.1587636	best: 0.1587636 (91000)	total: 8m 8s	remaining: 18m 42s
92000:	learn: 0.1055965	test: 0.1582771	best: 0.1582771 (92000)	total: 8m 14s	remaining: 18m 37s
93000:	learn: 0.1049509	test: 0.1577988	best: 0.1577988 (93000)	total: 8m 19s	remaining: 18m 31s
94000:	learn: 0.1043114	test: 0.1573102	best: 0.1573102 (94000)	total: 8m 24s	remaining: 18m 26s
95000:	learn: 0.1036788	test: 0.1568327	best: 0.1568327 (95000)	total: 8m 29s	remaining: 18m 20s
96000:	learn: 0.1030685	test: 0.1

169000:	learn: 0.0740513	test: 0.1358916	best: 0.1358916 (169000)	total: 15m 7s	remaining: 11m 43s
170000:	learn: 0.0737872	test: 0.1357222	best: 0.1357222 (170000)	total: 15m 12s	remaining: 11m 37s
171000:	learn: 0.0735262	test: 0.1355420	best: 0.1355420 (171000)	total: 15m 17s	remaining: 11m 32s
172000:	learn: 0.0732635	test: 0.1353692	best: 0.1353692 (172000)	total: 15m 23s	remaining: 11m 27s
173000:	learn: 0.0730077	test: 0.1352013	best: 0.1352013 (173000)	total: 15m 28s	remaining: 11m 21s
174000:	learn: 0.0727524	test: 0.1350374	best: 0.1350374 (174000)	total: 15m 33s	remaining: 11m 16s
175000:	learn: 0.0724980	test: 0.1348760	best: 0.1348760 (174999)	total: 15m 39s	remaining: 11m 10s
176000:	learn: 0.0722449	test: 0.1347100	best: 0.1347100 (176000)	total: 15m 44s	remaining: 11m 5s
177000:	learn: 0.0719947	test: 0.1345397	best: 0.1345397 (177000)	total: 15m 49s	remaining: 10m 59s
178000:	learn: 0.0717462	test: 0.1343783	best: 0.1343783 (178000)	total: 15m 54s	remaining: 10m 54s
17

252000:	learn: 0.0575978	test: 0.1255770	best: 0.1255770 (252000)	total: 22m 28s	remaining: 4m 16s
253000:	learn: 0.0574472	test: 0.1254869	best: 0.1254869 (252998)	total: 22m 34s	remaining: 4m 11s
254000:	learn: 0.0573003	test: 0.1254002	best: 0.1254002 (254000)	total: 22m 39s	remaining: 4m 6s
255000:	learn: 0.0571541	test: 0.1253136	best: 0.1253136 (255000)	total: 22m 44s	remaining: 4m
256000:	learn: 0.0570084	test: 0.1252271	best: 0.1252271 (256000)	total: 22m 50s	remaining: 3m 55s
257000:	learn: 0.0568621	test: 0.1251413	best: 0.1251413 (257000)	total: 22m 55s	remaining: 3m 50s
258000:	learn: 0.0567181	test: 0.1250564	best: 0.1250564 (258000)	total: 23m	remaining: 3m 44s
259000:	learn: 0.0565753	test: 0.1249747	best: 0.1249747 (259000)	total: 23m 5s	remaining: 3m 39s
260000:	learn: 0.0564334	test: 0.1248915	best: 0.1248915 (260000)	total: 23m 11s	remaining: 3m 34s
261000:	learn: 0.0562901	test: 0.1248071	best: 0.1248071 (261000)	total: 23m 16s	remaining: 3m 28s
262000:	learn: 0.056

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [1:22:04<54:43, 1641.72s/it]

0:	learn: 9.6167446	test: 9.6132904	best: 9.6132904 (0)	total: 6.4ms	remaining: 32m
1000:	learn: 0.8925633	test: 0.9062133	best: 0.9062133 (1000)	total: 5.31s	remaining: 26m 25s
2000:	learn: 0.6730065	test: 0.6877037	best: 0.6877037 (2000)	total: 10.6s	remaining: 26m 15s
3000:	learn: 0.5705066	test: 0.5873798	best: 0.5873798 (3000)	total: 15.9s	remaining: 26m 17s
4000:	learn: 0.5045427	test: 0.5234231	best: 0.5234231 (4000)	total: 21.2s	remaining: 26m 9s
5000:	learn: 0.4590736	test: 0.4796204	best: 0.4796204 (5000)	total: 26.6s	remaining: 26m 8s
6000:	learn: 0.4226166	test: 0.4446110	best: 0.4446110 (6000)	total: 31.9s	remaining: 26m 4s
7000:	learn: 0.3947487	test: 0.4180132	best: 0.4180132 (7000)	total: 37.3s	remaining: 26m 2s
8000:	learn: 0.3711844	test: 0.3956955	best: 0.3956955 (8000)	total: 42.8s	remaining: 26m 1s
9000:	learn: 0.3513296	test: 0.3769738	best: 0.3769738 (9000)	total: 48.1s	remaining: 25m 54s
10000:	learn: 0.3345036	test: 0.3612641	best: 0.3612641 (10000)	total: 53.5

In [33]:
cat_df = pd.read_csv('./submission_pseudo_labeling_v1.csv')
y_cat = cat_df[target[0]]
lgb_df = pd.read_csv('y_pred_lgb.csv')
y_lgb = lgb_df[target[0]]
xgb_df = pd.read_csv('./y_pred_xgb.csv')
y_xgb = xgb_df[target[0]]

In [34]:
res_df['contest-tmp2m-14d__tmp2m'] = (y_cat * 8 + y_lgb * 2 + y_xgb * 0) / 10
res_df.to_csv('submission_ensemble.csv', index=False)